In [1]:
model = "SSP_dev_SSP2_v0.1"
scenario = "baseline_v0.1_materials_no_pow_sect"


import ixmp
import message_ix
mp = ixmp.Platform("ixmp_dev")
#scen = message_ix.Scenario(mp, model, scenario)

<IPython.core.display.Javascript object>

In [5]:
from message_data.model.material import data_petro
import importlib

In [47]:
importlib.reload(data_petro)
df = data_petro.gen_mock_demand_petro(scen, 1, 1)
df_baseyear = df[(df["commodity"]=="HVC") & (df["year"]==2025)]

In [49]:
df

,node,commodity,level,year,time,value,unit
0,R12_AFR,HVC,demand,2030,year,3.257632,t
1,R12_CHN,HVC,demand,2030,year,0.614827,t
2,R12_EEU,HVC,demand,2030,year,3.551030,t
3,R12_FSU,HVC,demand,2030,year,6.284582,t
4,R12_LAM,HVC,demand,2030,year,13.712894,t
...,...,...,...,...,...,...,...
151,R12_PAO,HVC,demand,2025,year,11.000000,t
152,R12_PAS,HVC,demand,2025,year,37.500000,t
153,R12_RCPA,HVC,demand,2025,year,10.700000,t
154,R12_SAS,HVC,demand,2025,year,29.200000,t


In [37]:
scen.firstmodelyear

2025

In [112]:
df_gdp = scen.par("gdp_calibrate")

In [113]:
df_gdp = df_gdp.set_index("node")
df_gdp = df_gdp.join(df_gdp[df_gdp["year"]==2020]["value"], rsuffix="_2020")
df_gdp["growth"] = (df_gdp["value"] / df_gdp["value_2020"])-1

In [114]:
df_gdp

,year,value,unit,value_2020,growth
node,,,,,
R12_AFR,2020,998.98924,T$,998.98924,0.000000
R12_AFR,2025,1368.01863,T$,998.98924,0.369403
R12_AFR,2030,1856.87524,T$,998.98924,0.858754
R12_AFR,2035,2479.36677,T$,998.98924,1.481875
R12_AFR,2040,3314.60341,T$,998.98924,2.317957
...,...,...,...,...,...
R12_WEU,2070,38793.79891,T$,13849.19218,1.801160
R12_WEU,2080,45819.24718,T$,13849.19218,2.308442
R12_WEU,2090,52976.57491,T$,13849.19218,2.825247


In [92]:
df_gdp[df_gdp["year"]==2020]["value"]

,node,year,value,unit,growth_abs
0,R12_AFR,2020,998.98924,T$,NaN
140,R12_CHN,2020,12597.24955,T$,-84206.84352
14,R12_EEU,2020,1782.52848,T$,-68011.55969
98,R12_FSU,2020,2824.38360,T$,-5491.21456
28,R12_LAM,2020,4997.59636,T$,-13906.42139
42,R12_MEA,2020,3620.98548,T$,-45701.43853
56,R12_NAM,2020,18030.12282,T$,-30434.35428
112,R12_PAO,2020,4865.43755,T$,-50091.24951
126,R12_PAS,2020,4867.54648,T$,-8906.70881
154,R12_RCPA,2020,354.01999,T$,-43266.03017


In [84]:
#df_gdp_gro = pd.concat([df_gdp, ], axis=1)
df_gdp["growth_rel"] = df_gdp["growth_abs"].cumsum() / df_gdp["value"]

In [73]:
scen.par("gdp_calibrate")

,node,year,value,unit
0,R12_AFR,2020,998.98924,T$
1,R12_AFR,2025,1368.01863,T$
2,R12_AFR,2030,1856.87524,T$
3,R12_AFR,2035,2479.36677,T$
4,R12_AFR,2040,3314.60341,T$
...,...,...,...,...
163,R12_RCPA,2070,3360.28429,T$
164,R12_RCPA,2080,4304.25447,T$
165,R12_RCPA,2090,5373.51508,T$
166,R12_RCPA,2100,6453.46984,T$


In [65]:
df_gdp["value"].diff().shift(-1)

0        18.552009
1        55.053443
2        96.956958
3       139.034792
4       135.760735
          ...     
190    7025.448265
191    7157.327735
192    6695.183911
193       0.000000
194            NaN
Name: value, Length: 240, dtype: float64

In [119]:
from message_data.model.material.material_demand import refactor_mat_demand_calc

In [129]:
from message_ix_models.util import private_data_path
from message_data.model.material.data_aluminum import gen_mock_demand_aluminum

import importlib
importlib.reload(refactor_mat_demand_calc)

pop = scen.par("bound_activity_up", {"technology": "Population"})
pop = pop.loc[pop.year_act >= 2020].rename(
    columns={"year_act": "year", "value": "pop.mil", "node_loc": "region"}
)

# import pdb; pdb.set_trace()

pop = pop[["region", "year", "pop.mil"]]

base_demand = gen_mock_demand_aluminum(scen)
base_demand = base_demand.loc[base_demand.year == 2020].rename(
    columns={"value": "demand.tot.base", "node": "region"}
)

refactor_mat_demand_calc.derive_steel_demand(pop, base_demand, str(private_data_path("material")))

Index([   'Model', 'Scenario',   'Region', 'Variable',     'Unit',       2000,
             2005,       2010,       2015,       2020,       2025,       2030,
             2035,       2040,       2045,       2050,       2055,       2060,
             2070,       2080,       2090,       2100,       2110,    'Notes'],
      dtype='object')
Index(['TIMER Region',   'Unnamed: 1',            't',           1970,
                 1971,           1972,           1973,           1974,
                 1975,           1976,           1977,           1978,
                 1979,           1980,           1981,           1982,
                 1983,           1984,           1985,           1986,
                 1987,           1988,           1989,           1990,
                 1991,           1992,           1993,           1994,
                 1995,           1996,           1997,           1998,
                 1999,           2000,           2001,           2002,
                 2003,

KeyError: 'Reg_no'

In [24]:
from message_ix_models.util import (
    broadcast,
    make_io,
    make_matched_dfs,
    same_node,
    add_par_data,
    private_data_path,
)
import pandas as pd
df_gdp = pd.read_excel(
        private_data_path("material", "methanol", "methanol demand.xlsx"),
        sheet_name="GDP_baseline",
    )    
df = df_gdp[(~df_gdp["Region"].isna()) & (df_gdp["Region"] != "World")]
df = df.dropna(axis=1)
df

,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,...,2040,2045,2050,2055,2060,2070,2080,2090,2100,2110
0,ENGAGE_SSP2_v4.1.5,baseline,AFR,GDP|PPP,billion US$2010/yr OR local currency,1794.2820,2402.228030,3084.364057,4244.401513,5306.159830,...,8819.133966,11918.121112,14602.417875,19755.749042,23807.218699,37616.622442,56865.524017,81978.281459,112862.365070,155381.562316
1,ENGAGE_SSP2_v4.1.5,baseline,RCPA,GDP|PPP,billion US$2010/yr OR local currency,1079.8701,2150.548072,2373.445674,3270.479945,3893.061971,...,5073.624444,5537.560712,5964.216044,6245.074034,6503.801283,6880.935088,7096.848967,7207.835099,7292.012794,7377.173569
2,ENGAGE_SSP2_v4.1.5,baseline,EEU,GDP|PPP,billion US$2010/yr OR local currency,2076.9590,2618.822609,2730.706601,3143.074779,3501.253203,...,4256.420498,4597.068272,4910.855123,5246.396492,5553.862302,6242.035364,6847.727435,7413.134880,7998.343398,8629.749513
3,ENGAGE_SSP2_v4.1.5,baseline,FSU,GDP|PPP,billion US$2010/yr OR local currency,3162.3830,4356.569587,4731.600636,5781.870531,6651.451416,...,8506.190346,9251.644256,9950.008692,10722.125627,11446.507830,13192.598513,14735.329180,16214.272854,17702.925869,19328.254011
4,ENGAGE_SSP2_v4.1.5,baseline,LAM,GDP|PPP,billion US$2010/yr OR local currency,6463.3140,8526.634978,9420.268593,11361.550043,12958.770551,...,16842.884509,19144.971803,21230.787541,23785.112717,26129.992341,31454.646621,37181.975468,43231.355239,49620.005864,56952.759595
5,ENGAGE_SSP2_v4.1.5,baseline,MEA,GDP|PPP,billion US$2010/yr OR local currency,4119.7470,5191.844904,6351.518847,8052.544729,9560.925873,...,13412.104505,15685.103462,17790.488063,20402.696892,22849.032178,28824.680775,35379.951669,42596.412518,50429.418189,59702.826330
6,ENGAGE_SSP2_v4.1.5,baseline,NAM,GDP|PPP,billion US$2010/yr OR local currency,15903.0700,16473.205062,20642.913911,23092.253472,25377.748594,...,29535.387817,31456.430747,33290.073322,35187.978277,36999.868551,40983.108844,44752.779080,48349.422545,51744.756419,55378.527310
7,ENGAGE_SSP2_v4.1.5,baseline,PAO,GDP|PPP,billion US$2010/yr OR local currency,5323.9890,5498.664307,6170.836262,6626.157558,7058.364196,...,7782.799460,8172.236645,8539.725624,8980.617553,9391.395793,10315.595017,11207.031058,12090.062676,12969.968251,13913.912686
8,ENGAGE_SSP2_v4.1.5,baseline,PAS,GDP|PPP,billion US$2010/yr OR local currency,5112.7630,6593.728910,8074.694821,10087.804348,11864.037191,...,15989.457685,18306.797895,20390.026395,22741.413604,24982.135093,29828.339187,34700.948700,39529.321059,44257.183183,49550.516195
9,ENGAGE_SSP2_v4.1.5,baseline,SAS,GDP|PPP,billion US$2010/yr OR local currency,4943.5830,7639.789695,9198.303861,13133.949690,15990.168190,...,25016.096851,31679.807625,36496.175849,44208.145112,50386.897810,66407.736884,83868.117540,102061.793719,120825.538934,143038.940693


In [17]:
df_gdp.pivot(columns="year_act", values="value", index="node_loc")

year_act,1990,1995,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2070,2080,2090,2100,2110
node_loc,,,,,,,,,,,,,,,,,,,,
R12_AFR,271.455524,290.007532,345.060976,442.017934,581.052726,716.813461,998.989240,1368.018625,1856.875242,2479.366774,3314.603412,4517.417565,6162.000935,8577.989840,11963.585420,22747.571333,39895.853924,64039.060554,96804.093067,96804.093067
R12_CHN,838.663234,1455.044821,2160.256814,3396.324652,5703.315566,6612.838095,12597.249546,18796.728814,26265.311778,31779.168946,37291.085088,42057.027778,46300.743062,49633.467401,52750.003770,58298.326044,63001.761304,67276.308380,69794.088167,69794.088167
R12_EEU,890.917087,802.783706,944.330209,1164.365537,1356.751983,1429.632941,1782.528479,2135.375473,2527.594422,2908.785626,3307.539451,3673.647838,4045.608182,4427.270005,4850.211186,5775.938713,6672.119355,7552.177728,8315.598163,8315.598163
R12_FSU,1749.056217,1035.815476,1116.838624,1549.642196,1887.986772,2070.092516,2824.383598,3612.532110,4540.650530,5400.750295,6288.079443,7034.991188,7731.857143,8453.431718,9324.554152,11366.946565,13374.462312,15560.537726,17847.927374,18904.017751
R12_LAM,1777.475911,2104.621622,2448.478261,2794.058167,3428.029377,3752.743938,4997.596357,6313.315558,7870.347914,9439.545455,11247.304734,13141.528193,15271.185657,17543.266146,20056.958333,25680.038879,32385.211186,40462.277720,49322.424009,49322.424009
R12_MEA,923.149621,1139.373737,1518.708333,1872.857955,2347.592803,2728.016143,3620.985480,4689.533422,6001.189152,7468.361988,9180.558757,10973.157936,12956.405973,15113.972648,17571.820273,23368.410602,30275.401896,38534.015030,48464.477103,48464.477103
R12_NAM,8424.876209,9511.891683,11766.032882,13260.831721,13881.779497,15164.541706,18030.122824,20731.982116,23411.377937,25908.973396,28432.682303,30839.028757,33195.516022,35609.614390,38212.132723,43260.486550,47544.976471,51359.255294,54956.687059,54956.687059
R12_PAO,3232.875109,3491.610480,3743.429694,4056.182533,4197.181659,4444.424839,4865.437555,5328.230735,5790.865455,6207.936141,6620.625825,7084.795192,7566.369971,8111.292818,8719.640947,10185.261488,11671.063437,12839.161176,13774.255294,13774.255294
R12_PAS,984.333333,1414.102258,1631.705179,2061.975432,3085.776892,3667.170716,4867.546481,6274.462718,7885.997801,9667.563245,11716.739693,13845.204708,16171.420315,18335.382760,20622.422972,25656.708175,31310.156530,37491.349057,43620.050164,43620.050164


In [4]:
from message_data.model.material import data_methanol_new
import importlib
importlib.reload(data_methanol_new)

<module 'message_data.model.material.data_methanol_new' from 'C:\\Users\\maczek\\PycharmProjects\\message_data\\message_data\\model\\material\\data_methanol_new.py'>

In [5]:
meth_dict = data_methanol_new.gen_data_methanol_new(scen)

In [9]:
setlist = scen.set_list()

In [21]:
for par, df in meth_dict.items():
    for col in df.columns:
        df_set = list(df[col].unique())
        if col in ["technology", "commodity", "emission"]:
            continue
        print(col,":")
        if col in setlist:
            for _set in df_set:
                if _set not in list(scen.set(col)):
                    print(_set)
        print()

node :

level :
final_material

year :
time :

value :

unit :

node_loc :

year_vtg :

value :

unit :

node_loc :

year_vtg :

year_act :

value :

unit :

node_loc :

year_vtg :

year_act :

mode :
fuel
feedstock
ethanol

time :

value :

unit :

node_loc :

year_vtg :

year_act :

mode :
fuel
feedstock
ethanol

node_origin :

level :
primary_material
import_fs
export_fs
secondary_material
final_material
useful_petro
useful_resins

time :

time_origin :

value :

unit :

node_loc :

year_vtg :

year_act :

mode :
fuel
feedstock
ethanol

node_dest :

level :
primary_material
secondary_material
import_fs
export_fs
final_material

time :

time_dest :

value :

unit :

node_loc :

year_vtg :

value :

unit :

node_loc :

year_vtg :

value :

unit :

node_loc :

year_vtg :

year_act :

time :

value :

unit :

relation :
CO2_PtX_trans_disp_split
gas_util
SO2_red_synf
global_GSO2
global_GCO2
meth_exp_tot
OilPrices
PE_export_total
PE_import_total
PE_synliquids
PE_total_direct
PE_total_engi

In [25]:
relations = []
for par, df in meth_dict.items():
    if par != "relation_activity":
        continue
    for col in df.columns:
        df_set = list(df[col].unique())
        if col not in ["relation"]:
            continue
        print(col,":")
        if col in setlist:
            for _set in df_set:
                if _set not in list(scen.set(col)):
                    relations.append(_set)
        print()

relation :


In [33]:
import yaml

file_path = "C:/Users\maczek\PycharmProjects\message_data\data\material\set.yaml"
try:
    with open(file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)
        #print(yaml_data)
except FileNotFoundError:
    print("File not found.")
except yaml.YAMLError as e:
    print("Error reading YAML:", e)
for i in yaml_data.values():
    if "add" in i.keys():
        i["add"]
tec_list = {}
for k, v in list(yaml_data.items()):
    if "relation" in v.keys():
        if "add" in v["relation"].keys():
            tec_list[k] = v["relation"]["add"]
tec_list

{'common': {'commodity': {'require': ['coal', 'gas', 'electr', 'ethanol', 'methanol', 'fueloil', 'lightoil', 'hydrogen', 'lh2', 'd_heat'], 'add': ['biomass', 'water', 'fresh_water']}, 'level': {'require': ['primary', 'secondary', 'useful', 'final', 'water_supply', 'export', 'import']}, 'type_tec': {'add': ['industry']}, 'mode': {'add': ['M1', 'M2']}, 'emission': {'add': ['CO2', 'CH4', 'N2O', 'NOx', 'SO2', 'PM2p5', 'CF4']}, 'year': {'add': [1980, 1990, 2000, 2010]}, 'type_year': {'add': [2010]}}, 'generic': {'commodity': {'add': ['ht_heat', 'lt_heat']}, 'level': {'add': ['useful_steel', 'useful_cement', 'useful_aluminum', 'useful_refining', 'useful_petro', 'useful_resins']}, 'technology': {'add': ['furnace_foil_steel', 'furnace_loil_steel', 'furnace_biomass_steel', 'furnace_ethanol_steel', 'furnace_methanol_steel', 'furnace_gas_steel', 'furnace_coal_steel', 'furnace_elec_steel', 'furnace_h2_steel', 'hp_gas_steel', 'hp_elec_steel', 'fc_h2_steel', 'solar_steel', 'dheat_steel', 'furnace_fo

{'steel': ['minimum_recycling_steel',
  'max_global_recycling_steel',
  'max_regional_recycling_steel'],
 'aluminum': ['minimum_recycling_aluminum', 'maximum_recycling_aluminum'],
 'fertilizer': ['NH3_trd_cap', 'NFert_trd_cap'],
 'methanol': ['CO2_PtX_trans_disp_split', 'meth_exp_tot']}

In [34]:
tec_list_flat = [item for sublist in tec_list.values() for item in sublist]
tec_list_flat

['minimum_recycling_steel',
 'max_global_recycling_steel',
 'max_regional_recycling_steel',
 'minimum_recycling_aluminum',
 'maximum_recycling_aluminum',
 'NH3_trd_cap',
 'NFert_trd_cap',
 'CO2_PtX_trans_disp_split',
 'meth_exp_tot']

In [36]:
rels_depr = [i for i in relations if i not in tec_list_flat]

In [37]:
import yaml
file_path = 'missing_rels.yaml'

# Dump the list to the YAML file
with open(file_path, 'w') as file:
    yaml.dump(rels_depr, file, default_flow_style=False)

In [40]:
file.close()

In [41]:
file_path = "C:/Users\maczek\PycharmProjects\message_data\message_data\model\material\petrochemical model fixes notebooks\missing_rels.yaml"

with open(file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)
    
yaml_data

['gas_util',
 'SO2_red_synf',
 'global_GSO2',
 'global_GCO2',
 'OilPrices',
 'PE_export_total',
 'PE_import_total',
 'PE_synliquids',
 'PE_total_direct',
 'PE_total_engineering',
 'FE_transport',
 'SO2_red_ref',
 'trp_energy',
 'total_TCO2',
 'nuc_lc_in_el',
 'FE_liquids',
 'FE_final_energy']

In [2]:
df_scens = mp.scenario_list(model="MESSAGEix-Materials")

In [3]:
df_scens.sort_values("cre_date", ascending=False)

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
221,MESSAGEix-Materials,SSP_supply_cost_test_LED,MESSAGE,1,0,maczek,2023-11-27 01:43:09.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|SSP_s...,1
225,MESSAGEix-Materials,SSP_supply_cost_test_SSP4,MESSAGE,1,0,maczek,2023-11-27 01:24:36.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|SSP_s...,1
224,MESSAGEix-Materials,SSP_supply_cost_test_SSP3,MESSAGE,1,0,maczek,2023-11-27 01:17:11.000000,maczek,2023-11-27 01:24:20.000000,None,None,clone Scenario from 'MESSAGEix-Materials|SSP_s...,1
222,MESSAGEix-Materials,SSP_supply_cost_test_SSP1,MESSAGE,1,0,maczek,2023-11-27 00:59:57.000000,maczek,2023-11-27 01:07:41.000000,None,None,clone Scenario from 'MESSAGEix-Materials|SSP_s...,1
226,MESSAGEix-Materials,SSP_supply_cost_test_SSP5,MESSAGE,1,0,maczek,2023-11-27 00:54:14.000000,None,None,None,None,clone Scenario from 'MESSAGEix-Materials|SSP_s...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,MESSAGEix-Materials,NoPolicy,MESSAGE,1,0,unlu,2021-04-26 18:00:28.000000,maczek,2022-10-26 18:48:00.000000,None,None,clone Scenario from 'ENGAGE_SSP2_v4.1.5|baseli...,88
81,MESSAGEix-Materials,NoPolicy_all_with_power_sect_bldg,MESSAGE,1,0,min,2021-03-26 17:18:25.000000,min,2021-03-26 18:38:22.000000,None,None,clone Scenario from 'ENGAGE_SSP2_v4.1.4|baseli...,1
80,MESSAGEix-Materials,NoPolicy_all_with_power_sect,MESSAGE,1,0,min,2021-03-26 14:47:46.000000,None,None,None,None,clone Scenario from 'ENGAGE_SSP2_v4.1.4|baseli...,2
220,MESSAGEix-Materials,NoPolicypower,MESSAGE,1,0,min,2021-03-26 11:23:39.000000,None,None,None,None,clone Scenario from 'ENGAGE_SSP2_v4.1.4|baseli...,25


In [ ]:
clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12|baseline_DEFAULT', version: 21

In [138]:
#'MESSAGEix-GLOBIOM 1.1-R12|baseline_DEFAULT'
mp.scenario_list(model="MESSAGEix-GLOBIOM 1.1-R12")

,model,scenario,scheme,is_default,is_locked,cre_user,cre_date,upd_user,upd_date,lock_user,lock_date,annotation,version
0,MESSAGEix-GLOBIOM 1.1-R12,2degrees,MESSAGE,1,0,unlu,2023-10-09 11:41:56.000000,unlu,2023-10-09 12:06:23.000000,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
1,MESSAGEix-GLOBIOM 1.1-R12,MACRO_test,MESSAGE,1,0,min,2022-05-02 16:57:38.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
2,MESSAGEix-GLOBIOM 1.1-R12,base_GLOBIOM_noSDG,MESSAGE,1,0,min,2023-08-01 10:58:46.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,6
3,MESSAGEix-GLOBIOM 1.1-R12,base_GLOBIOM_noSDG_noConst,MESSAGE,1,0,min,2023-08-11 15:17:05.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
4,MESSAGEix-GLOBIOM 1.1-R12,base_GLOBIOM_update,MESSAGE,1,0,min,2023-04-18 14:25:06.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,3
5,MESSAGEix-GLOBIOM 1.1-R12,baseline_DEFAULT,MESSAGE,1,0,min,2023-07-27 10:24:37.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,21
6,MESSAGEix-GLOBIOM 1.1-R12,baseline_DEFAULT_2507,MESSAGE,1,0,unlu,2023-07-25 17:28:24.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
7,MESSAGEix-GLOBIOM 1.1-R12,baseline_DEFAULT_clone,MESSAGE,1,0,unlu,2023-02-01 09:55:44.000000,None,None,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
8,MESSAGEix-GLOBIOM 1.1-R12,baseline_DEFAULT_gdp_update,MESSAGE,1,0,min,2022-03-03 00:24:43.000000,min,2022-03-03 00:33:38.000000,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,25
9,MESSAGEix-GLOBIOM 1.1-R12,baseline_DEFULT_check,MESSAGE,1,0,unlu,2023-07-31 14:17:24.000000,unlu,2023-07-31 14:26:45.000000,None,None,clone Scenario from 'MESSAGEix-GLOBIOM 1.1-R12...,1
